# Spectral Clustering  

In this notebook, we will use cuGraph to identify the cluster in a test graph using the Spectral Modularity Maxunuzation algorith,  

Notebook Credits
* Original Authors: Bradley Rees and James Wyles
* Last Edit: 03/08/2019

RAPIDS Versions: 0.6.0

Test Hardware
* GP100 32G, CUDA 9,2

## Introduction
Spectral clustering uses the eigenvectors of a Laplacian of the input graph to find a 
given number of clusters which satisfy a given quality metric.
Modularity Maximization is such a metric. 

@See:  https://en.wikipedia.org/wiki/Spectral_clustering

To perform spectral clustering using the modularity maximization metric in cugraph use:
__cugraph.spectralModularityMaximizationClustering(G, num_clusters, num_eigen_vects)__

Input
* G: A cugraph.Graph object
* num_clusters: The number of clusters to find
* num_eig: (optional) The number of eigenvectors to use

Returns
* __df__: cudf.DataFrame with two names columns:
    * df["vertex"]: The vertex id.
    * df["cluster"]: The assigned partition.
        
        
        
 ## cuGraph 0.6 Notice 
cuGraph version 0.6 has some limitations:
* Only Int32 Vertex ID are supported
* Only float (FP32) edge data is supported
* Vertex numbering is assumed to start at zero

These limitations are being addressed and will be fixed future versions.  
These example notebooks will illustrate how to manipulate the data so that it comforms to the current limitations 


## References
https://research.nvidia.com/publication/2017-06_Parallel-Modularity-Clustering       


### Test Data
We will be using the Zachary Karate club dataset 
*W. W. Zachary, An information flow model for conflict and fission in small groups, Journal of
Anthropological Research 33, 452-473 (1977).*


![Karate Club](../img/zachary_black_lines.png)

Zachary used a min-cut flow model to partition the graph into two clusters, shown by the circles and squares.  Zarchary wanted just two cluster based on a conflict that caused the Karate club to break into two separate clubs.  Many social network clustering methods identify more that two social groups in the data.

In [1]:
# Import needed libraries
import cugraph
import cudf
import numpy as np
from collections import OrderedDict

## Read data using cuDF

In [2]:
# Test file  - using the clasic Karate club dataset.  
datafile='../data/networks/karate-data.csv'

# Read the data file
cols = ["src", "dst"]

dtypes = OrderedDict([
        ("src", "int32"), 
        ("dst", "int32")
        ])

gdf = cudf.read_csv(datafile, names=cols, delimiter='\t', dtype=list(dtypes.values()) )

In [3]:
# Need to shift the vertex IDs to start with zero rather than one (next version of cuGraph will fix this issue)
gdf["src_0"] = gdf["src"] - 1
gdf["dst_0"] = gdf["dst"] - 1

In [4]:
# The algorithm requires that there are edge weights.  In this case all the weights are being ste to 1
gdf["data"] = cudf.Series(np.ones(len(gdf), dtype=np.float32))

In [5]:
gdf.head().to_pandas()

,src,dst,src_0,dst_0,data
0,1,2,0,1,1.0
1,1,3,0,2,1.0
2,1,4,0,3,1.0
3,1,5,0,4,1.0
4,1,6,0,5,1.0


In [6]:
# create a Graph 
G = cugraph.Graph()
G.add_edge_list(gdf["src_0"], gdf["dst_0"], gdf["data"])

In [7]:
# Call spectralModularityMaximizationClustering on the graph for 3 clusters
# using 3 eigenvectors:
df = cugraph.spectralModularityMaximizationClustering(G, 3, num_eigen_vects=3)

In [8]:
# Check the modularity score for the produced clustering
score = cugraph.analyzeClustering_modularity(G, 2, df['cluster'])
score

0.13151457905769348

#### Define and print function, but adjust vertex ID so that they match the illustration

In [10]:
def print_cluster(df, id):
    
    # See which nodes are in cluster 0:
    part = []
    for i in range(len(df)):
        if (df['cluster'][i] == id):
            part.append(df['vertex'][i] +1)
    print(part)

In [11]:
# See which nodes are in cluster 0:
print_cluster(df, 0)

[26, 28, 29, 33]


In [12]:
print_cluster(df, 1)

[1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 17, 18, 20, 22]


In [13]:
print_cluster(df, 2)

[3, 15, 16, 19, 21, 23, 24, 25, 27, 30, 31, 32, 34]


Copyright (c) 2019, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___